# 1. Generate StyleGAN2 images
Generates random images from pretrained StyleGAN2 ffhq-config-f.

Details about configuration can be found in [official implemetation repository](https://github.com/NVlabs/stylegan2).

In [ ]:
from src.stylegan_generator import StyleGANGenerator
from tqdm import tqdm
from pathlib import Path

import cv2
import h5py
import numpy as np
import matplotlib.pyplot as plt

Generated 250 000 radnom images. To change to amount of images generated, change either **\_batch_size** or **\_n_batch**.

Number of generated images is therefore **\_batch_size** \* **\_n_batch**.

If you have more RAM available, we recommend to increase the **\_batch_size** (works fine/tested with 16GB RAM on 16 images per batch).

In [ ]:
_stylegan_ffhq_f_gdrive_url = 'https://nvlabs-fi-cdn.nvidia.com/stylegan2/networks/stylegan2-ffhq-config-f.pkl'

_image_size = (178, 218)
_img_dir = 'img/to_label/'
_noise_vector_dir = 'noise_vectors/'

_batch_size = 8
_n_batch = 32000

In [ ]:
Path(_img_dir).mkdir(parents=True, exist_ok=True)
Path(_noise_vector_dir).mkdir(parents=True, exist_ok=True)

## A) Generate images from scratch
Create directories and initialize SyleGAN2 generator.

In [ ]:
rnd = np.random.RandomState()
gen = StyleGANGenerator(_stylegan_ffhq_f_gdrive_url)

In [ ]:
z_list = []

for i in tqdm(range(_n_batch)):
    # for eatch batch generate _batch_size of random noise vectors
    latents = rnd.randn(_batch_size, *gen.Gs.input_shape[1:])
    z_list.append(latents) # append to the z_list for saving to HDD
    
    imgs = gen.get_images(latents) # generate _batch_size of images
    # resize images using lanczos interpolation
    imgs = np.asarray([cv2.resize(img, dsize=_image_size, interpolation=cv2.INTER_LANCZOS4) for img in imgs])
    
    # save each image to _img_dir
    for j, img in enumerate(imgs):
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR) # conversion from CV2 BGR default colors to RGB
        cv2.imwrite(_img_dir + 'img-{:06d}.png'.format(i*_batch_size+j), img)

Save and load saved noise vectors.

In [ ]:
z_concat = np.concatenate(z_list, axis=0)
z_concat.shape

In [ ]:
with h5py.File(_noise_vector_dir + 'sample_z.h5', 'w') as f:
    f.create_dataset('z', data=z_concat)

In [ ]:
with h5py.File(_noise_vector_dir + 'sample_z.h5', 'r') as f:
    z = f['z'][:]

In [ ]:
img = gen.get_images(np.reshape(z[44], (1,512)))
plt.imshow(img[0]/255)

## B) Generate images from downloaded noise vectors

Download file of 250 000 radnom noise vectors.

In [ ]:
! cd noise_vectors && wget -nc data.belanec.eu/noise_vectors/sample_z.h5

Open the file.

In [ ]:
with h5py.File(_noise_vector_dir + 'sample_z.h5', 'r') as f:
    z = f['z'][:]

In [ ]:
gen = StyleGANGenerator(_stylegan_ffhq_f_gdrive_url)

for i in tqdm(range(len(z))):
    # generate image
    img = gen.get_images(np.reshape(z[i], (1,512)))
    
    # resize images using lanczos interpolation
    img = cv2.resize(img[0], dsize=_image_size, interpolation=cv2.INTER_LANCZOS4)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR) # conversion from CV2 BGR default colors to RGB
    cv2.imwrite(_img_dir + 'img-{:06d}.png'.format(i), img) # save image

## Next notebook
Continue to 02_classifiers (we recommend to start with InceptionV3 classifier).